In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True
The upload_fileobj method accepts a readable file-like object. The file object must be opened in binary mode, not text mode.

s3 = boto3.client('s3')
with open("FILE_NAME", "rb") as f:
    s3.upload_fileobj(f, "BUCKET_NAME", "OBJECT_NAME")

In [37]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

ModuleNotFoundError: No module named 'pyaudio'

In [2]:
ACCESS_KEY = ""
SECRET_KEY = ""

In [3]:
def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='mp4',
        LanguageCode='en-US'
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                print(
                    f"Download the transcript from\n"
                    f"\t{job['TranscriptionJob']['Transcript']['TranscriptFileUri']}.")
                return job['TranscriptionJob']['Transcript']
            #break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)

In [4]:
transcribe_client = boto3.client('transcribe',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY, region_name='us-east-2')
#    aws_session_token=SESSION_TOKEN)
file_uri = 's3://test-jean/query.mp4'
res = transcribe_file('test-query', file_uri, transcribe_client) 

ConflictException: An error occurred (ConflictException) when calling the StartTranscriptionJob operation: The requested job name already exists. Use a different job name.

In [5]:
result = transcribe_client.get_transcription_job(
    TranscriptionJobName='test-query'
) 

In [6]:
import json
import requests
res = requests.get(result['TranscriptionJob']['Transcript']['TranscriptFileUri']).text

In [7]:
dict(json.loads(res))['results']['transcripts'][0]['transcript']

'get first name in customer.'

In [8]:
import os
#os.popen('docker run --name valuenet -d -p 5050:5050 paulfitz/valuenet').read()
os.popen('docker start valuenet').read()

'valuenet\n'

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

print(mydb)

In [ ]:
import datetime
import csv


mycursor = mydb.cursor()
mycursor.execute("USE AgilOne")
mycursor.execute("DESCRIBE Customers")
x = list(mycursor)
f = open('Customers.csv', 'w')
c = csv.writer(f)
c.writerow([x[i][0] for i in range(len(x))])
mycursor.execute("SELECT * FROM Customers LIMIT 1")
x = list(mycursor)
rows = [[x[i][j].strftime("%D:%M:%Y") if isinstance(x[i][j], datetime.date) else x[i][j] for j in range(len(x[i]))] for i in range(len(x))]
for r in rows:
    c.writerow(r)
f.close()

In [16]:
# convert to SQL
result = os.popen('curl -F "csv=@Customers.csv" -F "q=how many First Names in Customers?" localhost:5050').read()

In [27]:
result = json.loads(result)

In [34]:
polly_client = boto3.Session(
                aws_access_key_id=ACCESS_KEY,                     
    aws_secret_access_key=SECRET_KEY,
    region_name='us-east-2').client('polly')

response = polly_client.synthesize_speech(VoiceId='Joanna',
                OutputFormat='mp3', 
                Text = 'The ansewer to your query is: %s' % result['result']['answer'][0][0])
file = open('speech.mp3', 'wb')
file.write(response['AudioStream'].read())
file.close()

In [35]:
from pygame import mixer

mixer.init()
mixer.music.load('speech.mp3')
mixer.music.play()